# Poutyne's Tips and Tricks

Poutyne offers a variety of tools for fine-tuning the information generated during the training, such as colouring the training update message, a progress bar, multi-GPUs, user callbacks interface and a user naming interface for the metrics' names. 

Let's install the latest version of Poutyne and colorama (if they are not already), and import all the needed packages.

In [1]:
%pip install --upgrade poutyne
%pip install --upgrade colorama
%matplotlib inline
import os
import math

import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import roc_auc_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import random_split, DataLoader
from torchvision import transforms, utils
from torchvision.datasets.mnist import MNIST

from poutyne import set_seeds, Model, ModelCheckpoint, CSVLogger, Callback, Experiment, SKLearnMetrics

## Hyperparameters, Dataset and Network

In this section, we setup the hyperparameters, dataset and network we will use throughout these tips and tricks. 

### Training Constants

Now, let's set our training constants. We first have the CUDA device used for training if one is present. Second, we set the `train_split` to 0.8 (80%) to use 80% of the dataset for training and 20% for testing the trained model. Third, we set the number of classes (i.e. one for each digit). Finally, we set the batch size (i.e. the number of elements to see before updating the model), the learning rate for the optimizer, and the number of epochs (i.e. the number of times we see the full dataset).

In [2]:
cuda_device = 0
device = torch.device("cuda:%d" % cuda_device if torch.cuda.is_available() else "cpu")

train_split_percent = 0.8

num_classes = 10

batch_size = 32
learning_rate = 0.1
num_epochs = 5

In Poutyne, as we will see in the following sections, you can define your own loss functions and optimizers. However, we can also pass magic strings to use PyTorch's standard optimizers and loss functions. Furthermore, for the optimizer, we can also use a dictionary to set other parameters as the learning rate, for instance, if we don't want the default learning rate.

Here, we initialize the dictionary for our optimizer as well as the string for our loss function. We thus use SGD with the specified learning rate and the cross-entropy loss.

In [3]:
optimizer = dict(optim='sgd', lr=learning_rate) # Could be 'sgd' if we didn't need to change the learning rate.
loss_function = 'cross_entropy'

### Loading the Dataset

The following code helps load the MNIST dataset and creates the PyTorch DataLoaders that split our datasets into batches. Then, the train DataLoader shuffles the examples of the training dataset to draw the examples without replacement.

In [4]:
full_train_dataset = MNIST('./datasets', train=True, download=True, transform=transforms.ToTensor())
test_dataset = MNIST('./datasets', train=False, download=True, transform=transforms.ToTensor())

num_data = len(full_train_dataset)
train_length = int(math.floor(train_split_percent * num_data))
valid_length = num_data - train_length

train_dataset, valid_dataset = random_split(full_train_dataset, 
                                            [train_length, valid_length],
                                            generator=torch.Generator().manual_seed(42))

train_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=2, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=2)

In [5]:
len(train_dataset), len(valid_dataset)

(48000, 12000)

### Initializing the Network

We create a fonction to initialize a simple convolutional neural network.

In [6]:
def create_network():
    return nn.Sequential(
        nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2),
        nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2),
        nn.Dropout(0.25),
        nn.Flatten(),
        nn.Linear(32*7*7, 128),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(128, num_classes)
    )

## Vanilla Usage

The following code trains our network in the simplest way possible with Poutyne. We use the accuracy metric so that we can see the performance during training.

In [7]:
# Instantiating our network
network = create_network()

# Poutyne Model on GPU
model = Model(network, optimizer, loss_function,
              batch_metrics=['accuracy'], 
              device=device)

# Train
model.fit_generator(train_loader, valid_loader, epochs=num_epochs)

# Test
test_loss, test_acc = model.evaluate_generator(test_loader)

Epoch: 1/5 Train steps: 1500 Val steps: 375 6.26s loss: 0.364863 acc: 88.131250 val_loss: 0.108600 val_acc: 96.633333
Epoch: 2/5 Train steps: 1500 Val steps: 375 6.29s loss: 0.126696 acc: 95.991667 val_loss: 0.063045 val_acc: 98.141667
Epoch: 3/5 Train steps: 1500 Val steps: 375 6.14s loss: 0.099989 acc: 96.943750 val_loss: 0.054086 val_acc: 98.433333
Epoch: 4/5 Train steps: 1500 Val steps: 375 6.16s loss: 0.083581 acc: 97.500000 val_loss: 0.051291 val_acc: 98.500000
Epoch: 5/5 Train steps: 1500 Val steps: 375 6.35s loss: 0.075759 acc: 97.762500 val_loss: 0.044111 val_acc: 98.775000
Test steps: 313 0.75s test_loss: 0.027585 test_acc: 98.970000                                                  


## Initilalizing Your Optimizer and Loss Function Yourself

Instead of using magic strings for the optimizer and the loss function, it's quite easy to initialize your own and pass them to Poutyne.

In [8]:
# Instantiating our network
network = create_network()

# Instantiating our loss function and optimizer
own_optimizer = optim.SGD(network.parameters(), lr=learning_rate)
own_loss_function = nn.CrossEntropyLoss()

# Poutyne Model on GPU
model = Model(network, own_optimizer, own_loss_function, 
              batch_metrics=['accuracy'], 
              device=device)

# Train
model.fit_generator(train_loader, valid_loader, epochs=num_epochs)

# Test
test_loss, test_acc = model.evaluate_generator(test_loader)

Epoch: 1/5 Train steps: 1500 Val steps: 375 6.31s loss: 0.378764 acc: 87.666667 val_loss: 0.089832 val_acc: 97.341667
Epoch: 2/5 Train steps: 1500 Val steps: 375 6.32s loss: 0.128875 acc: 96.127083 val_loss: 0.066408 val_acc: 98.033333
Epoch: 3/5 Train steps: 1500 Val steps: 375 6.38s loss: 0.098024 acc: 97.012500 val_loss: 0.054016 val_acc: 98.508333
Epoch: 4/5 Train steps: 1500 Val steps: 375 6.22s loss: 0.082791 acc: 97.452083 val_loss: 0.052957 val_acc: 98.441667
Epoch: 5/5 Train steps: 1500 Val steps: 375 6.37s loss: 0.075380 acc: 97.754167 val_loss: 0.050414 val_acc: 98.475000
Test steps: 313 0.75s test_loss: 0.036658 test_acc: 98.780000                                                  


## Bypassing PyTorch DataLoaders

Above, we defined DataLoaders for our datasets. However, with Poutyne, it is not strictly necessary since it provides the [`fit_dataset`](https://poutyne.org/model.html#poutyne.Model.fit_dataset) and [`evaluate_dataset`](https://poutyne.org/model.html#poutyne.Model.evaluate_dataset) methods to which you can pass the necessary parameters such as the batch size. Under the hood, Poutyne initializes the DataLoaders for you.

In [9]:
# Instantiating our network
network = create_network()

# Poutyne Model on GPU
model = Model(network, optimizer, loss_function,
              batch_metrics=['accuracy'], 
              device=device)

# Train
model.fit_dataset(train_dataset, 
                  valid_dataset, 
                  epochs=num_epochs, 
                  batch_size=batch_size, 
                  num_workers=2)

# Test
test_loss, test_acc = model.evaluate_dataset(test_dataset, 
                                             batch_size=batch_size, 
                                             num_workers=2)

Epoch: 1/5 Train steps: 1500 Val steps: 375 6.07s loss: 0.384156 acc: 87.412500 val_loss: 0.102481 val_acc: 96.950000
Epoch: 2/5 Train steps: 1500 Val steps: 375 6.31s loss: 0.128977 acc: 96.050000 val_loss: 0.066917 val_acc: 98.108333
Epoch: 3/5 Train steps: 1500 Val steps: 375 6.28s loss: 0.100961 acc: 96.966667 val_loss: 0.053764 val_acc: 98.525000
Epoch: 4/5 Train steps: 1500 Val steps: 375 6.20s loss: 0.086132 acc: 97.368750 val_loss: 0.050919 val_acc: 98.466667
Epoch: 5/5 Train steps: 1500 Val steps: 375 6.16s loss: 0.073995 acc: 97.766667 val_loss: 0.043581 val_acc: 98.791667
Test steps: 313 0.75s test_loss: 0.036267 test_acc: 98.980000                                                   


## Using Callbacks

One nice feature of Poutyne is [callbacks](https://poutyne.org/callbacks.html). Callbacks allow doing actions during the training of the neural network. In the following example, we use three callbacks. The first that saves the latest weights in a file to be able to continue the optimization at the end of training if more epochs are needed. The second that saves the best weights according to the performance on the validation dataset. The last that saves the displayed logs into a TSV file.

In [10]:
# Saves everything into saves/convnet_mnist
save_path = "saves/convnet_mnist"
os.makedirs(save_path, exist_ok=True)

callbacks = [
    # Save the latest weights to be able to continue the optimization at the end for more epochs.
    ModelCheckpoint(os.path.join(save_path, 'last_epoch.ckpt')),

    # Save the weights in a new file when the current model is better than all previous models.
    ModelCheckpoint(os.path.join(save_path, 'best_epoch_{epoch}.ckpt'), monitor='val_acc', mode='max', 
                    save_best_only=True, restore_best=True, verbose=True),

    # Save the losses and accuracies for each epoch in a TSV.
    CSVLogger(os.path.join(save_path, 'log.tsv'), separator='\t'),
]

In [11]:
# Instantiating our network
network = create_network()

# Poutyne Model on GPU
model = Model(network, optimizer, loss_function, 
              batch_metrics=['accuracy'],
              device=device)

# Train
model.fit_generator(train_loader,
                    valid_loader,
                    epochs=num_epochs,
                    callbacks=callbacks)

# Test
test_loss, test_acc = model.evaluate_generator(test_loader)

Epoch: 1/5 Train steps: 1500 Val steps: 375 6.06s loss: 0.363334 acc: 88.272917 val_loss: 0.083082 val_acc: 97.508333
Epoch 1: val_acc improved from -inf to 97.50833, saving file to saves/convnet_mnist/best_epoch_1.ckpt
Epoch: 2/5 Train steps: 1500 Val steps: 375 6.23s loss: 0.129105 acc: 96.068750 val_loss: 0.058399 val_acc: 98.166667
Epoch 2: val_acc improved from 97.50833 to 98.16667, saving file to saves/convnet_mnist/best_epoch_2.ckpt
Epoch: 3/5 Train steps: 1500 Val steps: 375 6.24s loss: 0.097697 acc: 97.066667 val_loss: 0.058446 val_acc: 98.316667
Epoch 3: val_acc improved from 98.16667 to 98.31667, saving file to saves/convnet_mnist/best_epoch_3.ckpt
Epoch: 4/5 Train steps: 1500 Val steps: 375 6.24s loss: 0.084468 acc: 97.452083 val_loss: 0.048789 val_acc: 98.550000
Epoch 4: val_acc improved from 98.31667 to 98.55000, saving file to saves/convnet_mnist/best_epoch_4.ckpt
Epoch: 5/5 Train steps: 1500 Val steps: 375 6.26s loss: 0.072188 acc: 97.812500 val_loss: 0.042376 val_acc: 

## Making Your Own Callback

While Poutyne provides a great number of [predefined callbacks](https://poutyne.org/callbacks.html), it is sometimes useful to make your own callback.

In the following example, we want to see the effect of temperature on the optimization of our neural network. To do so, we either increase or decrease the temperature during the optimization. As one can see in the result, temperature either as no effect or has a detrimental effect on the performance of the neural network. This is so because the temperature has for effect to artificially changing the learning rates. Since we have found the right learning rate, increasing or decreasing, it shows no improvement on the results.

In [12]:
class CrossEntropyLossWithTemperature(nn.Module):
    """
    This loss module is the cross-entropy loss function
    with temperature. It divides the logits by a temperature
    value before computing the cross-entropy loss.

    Args:
        initial_temperature (float): The initial value of the temperature.
    """

    def __init__(self, initial_temperature):
        super().__init__()
        self.temperature = initial_temperature
        self.celoss = nn.CrossEntropyLoss()

    def forward(self, y_pred, y_true):
        y_pred = y_pred / self.temperature
        return self.celoss(y_pred, y_true)

In [13]:
class TemperatureCallback(Callback):
    """
    This callback multiply the loss temperature with a decay before
    each batch.

    Args:
        celoss_with_temp (CrossEntropyLossWithTemperature): the loss module.
        decay (float): The value of the temperature decay.
    """
    def __init__(self, celoss_with_temp, decay):
        super().__init__()
        self.celoss_with_temp = celoss_with_temp
        self.decay = decay

    def on_train_batch_begin(self, batch, logs):
        self.celoss_with_temp.temperature *= self.decay

So our loss function will be the cross-entropy with temperature with an initial temperature of `0.1` and a temperature decay of `1.0008`.

In [14]:
custom_loss_function = CrossEntropyLossWithTemperature(0.1)
callbacks = [TemperatureCallback(custom_loss_function, 1.0008)]

Now let's test our training loop for one epoch using the accuracy as the batch metric.

In [15]:
# Instantiating our network
network = create_network()

# Poutyne Model on GPU
model = Model(network, optimizer, custom_loss_function, 
              batch_metrics=['accuracy'],
              device=device)

# Train
model.fit_generator(train_loader,
                    valid_loader,
                    epochs=num_epochs,
                    callbacks=callbacks)

# Test
test_loss, test_acc = model.evaluate_generator(test_loader)

Epoch: 1/5 Train steps: 1500 Val steps: 375 6.22s loss: 0.524956 acc: 82.987500 val_loss: 0.082677 val_acc: 97.525000
Epoch: 2/5 Train steps: 1500 Val steps: 375 6.17s loss: 0.157685 acc: 95.247917 val_loss: 0.064936 val_acc: 98.066667
Epoch: 3/5 Train steps: 1500 Val steps: 375 6.40s loss: 0.130516 acc: 96.093750 val_loss: 0.061940 val_acc: 98.200000
Epoch: 4/5 Train steps: 1500 Val steps: 375 6.27s loss: 0.130717 acc: 96.310417 val_loss: 0.064953 val_acc: 98.075000
Epoch: 5/5 Train steps: 1500 Val steps: 375 6.41s loss: 0.145397 acc: 95.914583 val_loss: 0.081095 val_acc: 97.983333
Test steps: 313 0.74s test_loss: 0.069856 test_acc: 98.240000                                                   


## Using Experiment

Most of the time, when using Poutyne (or even Pytorch in general), we will find ourselves in an iterative model hyperparameters finetuning loop. For efficient model search, we will usually wish to save our best performing models, their training and testing statistics and even sometimes wish to retrain an already trained model for further tuning. All of the above can be easily implemented with the flexibility of Poutyne Callbacks, but having to define and initialize each and every Callback object we wish for our model quickly feels cumbersome.

This is why Poutyne provides an [Experiment class](https://poutyne.org/experiment.html), which aims specifically at enabling quick model iteration search, while not sacrificing the quality of a single experiment - statistics logging, best models saving, etc. Experiment is actually a simple wrapper between a PyTorch network and Poutyne's core Callback objects for logging and saving. Given a working directory where to output the various logging files and a PyTorch network, the Experiment class reduces the whole training loop to a single line.

The following code uses [Poutyne's Experiment class](https://poutyne.org/experiment.html) to train a network for 5 epochs. The code is quite simpler than the code in the Poutyne Callbacks section while doing more (only a few lines). Once trained for 5 epochs, it is then possible to resume the optimization at the 5th epoch for 5 more epochs until the 10th epoch using the same function.

In [16]:
def experiment_train(network, name, epochs=5):
    """
    This function creates a Poutyne Experiment, trains the input module
    on the train loader and then tests its performance on the test loader.
    All training and testing statistics are saved, as well as best model
    checkpoints.
    
    Args:
        network (torch.nn.Module): The neural network to train.
        working_directory (str): The directory where to output files to save.
        epochs (int): The number of epochs. (Default: 5)
    """
    # Everything is going to be saved in ./saves/{name}.
    save_path = os.path.join('saves', name)

    # Poutyne Experiment
    expt = Experiment(save_path, network, optimizer=optimizer, task='classif', device=device)

    # Train
    expt.train(train_loader, valid_loader, epochs=epochs)

    # Test
    expt.test(test_loader)

In [17]:
network = create_network()
experiment_train(network, 'convnet_mnist_experiment', epochs=5)

Loading weights from saves/convnet_mnist_experiment/checkpoint.ckpt and starting at epoch 11.
Loading optimizer state from saves/convnet_mnist_experiment/checkpoint.optim and starting at epoch 11.
Restoring model from saves/convnet_mnist_experiment/checkpoint_epoch_10.ckpt
Found best checkpoint at epoch: 10
lr: 0.1, loss: 0.0380142, acc: 98.8438, fscore_micro: 0.988437, val_loss: 0.0415888, val_acc: 98.9167, val_fscore_micro: 0.989167
Loading checkpoint saves/convnet_mnist_experiment/checkpoint_epoch_10.ckpt
Running test
Test steps: 313 0.94s test_loss: 0.032788 test_acc: 98.980000 test_fscore_micro: 0.989800             


Notice how setting `task='classif'` when instantiating `Experiment` adds for use our loss function, the batch metric accuracy, the epoch metric F1 and set up callbacks that use them. If you wish, you still can use your own loss function and metrics instead of passing this argument.

We have trained for 5 epochs, let's now resume training for another 5 epochs for a total of 10 epochs. Notice that we reinstantiate the network. Experiment will load back the weights for us and resume training.

In [18]:
network = create_network()
experiment_train(network, 'convnet_mnist_experiment', epochs=10)

Loading weights from saves/convnet_mnist_experiment/checkpoint.ckpt and starting at epoch 11.
Loading optimizer state from saves/convnet_mnist_experiment/checkpoint.optim and starting at epoch 11.
Restoring model from saves/convnet_mnist_experiment/checkpoint_epoch_10.ckpt
Found best checkpoint at epoch: 10
lr: 0.1, loss: 0.0380142, acc: 98.8438, fscore_micro: 0.988437, val_loss: 0.0415888, val_acc: 98.9167, val_fscore_micro: 0.989167
Loading checkpoint saves/convnet_mnist_experiment/checkpoint_epoch_10.ckpt
Running test
Test steps: 313 0.88s test_loss: 0.032788 test_acc: 98.980000 test_fscore_micro: 0.989800             


## Coloring

Also, Poutyne use by default a coloring template of the training step when the package `colorama` is installed.
One could either remove the coloring (`progress_options=dict(coloring=False)`) or set a different coloring template using the fields:
`text_color`, `ratio_color`, `metric_value_color`, `time_color` and `progress_bar_color`.
If a field is not specified, the default color will be used.
[See available colors in colorama's source code](https://github.com/tartley/colorama/blob/9946cfb/colorama/ansi.py#L49).

Here an example where we set the `text_color` to RED and the `progress_bar_color` to LIGHTGREEN_EX.

In [19]:
progress_options = dict(
    coloring=dict(text_color="RED", progress_bar_color="LIGHTGREEN_EX")
)

In [20]:
# Instantiating our network
network = create_network()

# Poutyne Model on GPU
model = Model(network, optimizer, loss_function, 
              batch_metrics=['accuracy'],
              device=device)

# Train
model.fit_generator(train_loader, 
                    valid_loader, 
                    epochs=num_epochs, 
                    progress_options=progress_options)

# Test
test_loss, test_acc = model.evaluate_generator(test_loader, 
                                               progress_options=progress_options)

Epoch: 1/5 Train steps: 1500 Val steps: 375 6.34s loss: 0.372919 acc: 87.906250 val_loss: 0.087468 val_acc: 97.325000
Epoch: 2/5 Train steps: 1500 Val steps: 375 6.28s loss: 0.131601 acc: 96.054167 val_loss: 0.063087 val_acc: 98.150000
Epoch: 3/5 Train steps: 1500 Val steps: 375 6.16s loss: 0.101879 acc: 96.877083 val_loss: 0.054765 val_acc: 98.341667
Epoch: 4/5 Train steps: 1500 Val steps: 375 6.17s loss: 0.084397 acc: 97.497917 val_loss: 0.051853 val_acc: 98.375000
Epoch: 5/5 Train steps: 1500 Val steps: 375 6.34s loss: 0.075443 acc: 97.677083 val_loss: 0.048552 val_acc: 98.550000
Test steps: 313 0.76s test_loss: 0.035207 test_acc: 98.770000                                                   


## Epoch Metrics

It's also possible to used epoch metrics such as [`F1-score`](https://poutyne.org/metrics.html#poutyne.FBeta). You could also define your own epoch metric using the [`EpochMetric`](https://poutyne.org/metrics.html#epoch-metric-interface) interface.

In [21]:
# Instantiating our network
network = create_network()

# Poutyne Model on GPU
model = Model(network, optimizer, loss_function,
              batch_metrics=['accuracy'], 
              epoch_metrics=['f1'], 
              device=device)

# Train
model.fit_generator(train_loader, valid_loader, epochs=num_epochs)

# Test
test_loss, (test_acc, test_f1) = model.evaluate_generator(test_loader)

Epoch: 1/5 Train steps: 1500 Val steps: 375 8.25s loss: 0.375131 acc: 87.710417 fscore_micro: 0.877104 val_loss: 0.087440 val_acc: 97.250000 val_fscore_micro: 0.972500
Epoch: 2/5 Train steps: 1500 Val steps: 375 7.98s loss: 0.129126 acc: 96.089583 fscore_micro: 0.960896 val_loss: 0.063637 val_acc: 98.125000 val_fscore_micro: 0.981250
Epoch: 3/5 Train steps: 1500 Val steps: 375 8.16s loss: 0.098845 acc: 97.070833 fscore_micro: 0.970708 val_loss: 0.049600 val_acc: 98.566667 val_fscore_micro: 0.985667
Epoch: 4/5 Train steps: 1500 Val steps: 375 8.16s loss: 0.084809 acc: 97.383333 fscore_micro: 0.973833 val_loss: 0.046043 val_acc: 98.583333 val_fscore_micro: 0.985833
Epoch: 5/5 Train steps: 1500 Val steps: 375 8.06s loss: 0.073265 acc: 97.758333 fscore_micro: 0.977583 val_loss: 0.044262 val_acc: 98.708333 val_fscore_micro: 0.987083
Test steps: 313 0.95s test_loss: 0.032165 test_acc: 98.860000 test_fscore_micro: 0.988600             


Furthermore, you could also use the [`SKLearnMetrics`](https://poutyne.org/metrics.html#poutyne.SKLearnMetrics) wrapper to wrap a Scikit-learn metric as an epoch metric. Below, we show how to compute the AUC ROC using the [`SKLearnMetrics`](https://poutyne.org/metrics.html#poutyne.SKLearnMetrics) class.

In [22]:
def softmax(x, axis=1):
    """
    Compute softmax function.
    """
    e_x = np.exp(x - x.max(axis=axis, keepdims=True))
    return e_x / e_x.sum(axis=axis, keepdims=True)

def roc_auc(y_true, y_pred, **kwargs):
    """
    Since the `roc_auc_score` from Scikit-learn requires normalized probabilities,
    we use the softmax function on the predictions.
    """
    y_pred = softmax(y_pred)
    return roc_auc_score(y_true, y_pred, **kwargs)

# kwargs are keyword arguments we wish to pass to roc_auc.
roc_epoch_metric = SKLearnMetrics(roc_auc, 
                                  kwargs=dict(multi_class='ovr', average='macro'))

In [23]:
# Instantiating our network
network = create_network()

# Poutyne Model on GPU
model = Model(network, optimizer, loss_function,
              batch_metrics=['accuracy'], 
              epoch_metrics=['f1', roc_epoch_metric], 
              device=device)

# Train
model.fit_generator(train_loader, valid_loader, epochs=num_epochs)

# Test
test_loss, (test_acc, test_f1, test_roc) = model.evaluate_generator(test_loader)

Epoch: 1/5 Train steps: 1500 Val steps: 375 8.52s loss: 0.377142 acc: 87.545833 fscore_micro: 0.875458 roc_auc: 0.991092 val_loss: 0.091776 val_acc: 97.300000 val_fscore_micro: 0.973000 val_roc_auc: 0.999393
Epoch: 2/5 Train steps: 1500 Val steps: 375 8.47s loss: 0.127960 acc: 96.187500 fscore_micro: 0.961875 roc_auc: 0.998650 val_loss: 0.062178 val_acc: 98.125000 val_fscore_micro: 0.981250 val_roc_auc: 0.999660
Epoch: 3/5 Train steps: 1500 Val steps: 375 8.44s loss: 0.097671 acc: 97.104167 fscore_micro: 0.971042 roc_auc: 0.999189 val_loss: 0.054568 val_acc: 98.300000 val_fscore_micro: 0.983000 val_roc_auc: 0.999761
Epoch: 4/5 Train steps: 1500 Val steps: 375 8.54s loss: 0.084467 acc: 97.433333 fscore_micro: 0.974333 roc_auc: 0.999361 val_loss: 0.056364 val_acc: 98.366667 val_fscore_micro: 0.983667 val_roc_auc: 0.999744
Epoch: 5/5 Train steps: 1500 Val steps: 375 8.56s loss: 0.072830 acc: 97.810417 fscore_micro: 0.978104 roc_auc: 0.999525 val_loss: 0.046075 val_acc: 98.658333 val_fscor

## Custom Metric Names

It's also possible to name the metric using a tuple format `(<metric name>, metric)`. That way, it's possible to use multiple times the same metric type (i.e. having micro and macro F1-score).

In [24]:
# Instantiating our network
network = create_network()

# Poutyne Model on GPU
model = Model(network, optimizer, loss_function,
              batch_metrics=[("My accuracy name", 'accuracy')],
              epoch_metrics=[("My f1 name", 'f1')],
              device=device)

# Train
model.fit_generator(train_loader, valid_loader, epochs=num_epochs)

# Test
test_loss, (test_acc, test_f1) = model.evaluate_generator(test_loader)

Epoch: 1/5 Train steps: 1500 Val steps: 375 7.91s loss: 0.354099 My accuracy name: 88.662500 My f1 name: 0.886625 val_loss: 0.086341 val_My accuracy name: 97.300000 val_My f1 name: 0.973000
Epoch: 2/5 Train steps: 1500 Val steps: 375 7.68s loss: 0.125351 My accuracy name: 96.183333 My f1 name: 0.961833 val_loss: 0.057793 val_My accuracy name: 98.266667 val_My f1 name: 0.982667
Epoch: 3/5 Train steps: 1500 Val steps: 375 7.91s loss: 0.097752 My accuracy name: 97.050000 My f1 name: 0.970500 val_loss: 0.052885 val_My accuracy name: 98.500000 val_My f1 name: 0.985000
Epoch: 4/5 Train steps: 1500 Val steps: 375 8.08s loss: 0.083571 My accuracy name: 97.454167 My f1 name: 0.974542 val_loss: 0.050879 val_My accuracy name: 98.475000 val_My f1 name: 0.984750
Epoch: 5/5 Train steps: 1500 Val steps: 375 8.03s loss: 0.073961 My accuracy name: 97.739583 My f1 name: 0.977396 val_loss: 0.043142 val_My accuracy name: 98.700000 val_My f1 name: 0.987000
Test steps: 313 0.99s test_loss: 0.031091 test_My 

## Multi-GPUs

Finally, it's also possible to use multi-GPUs for your training either by specifying a list of devices or using the arg `"all"` to take them all.

> Obviously, you need more than one GPUs for that option.

In our case here, multi-gpus takes more time because the task is not big enough to profit from multi-gpus.

In [25]:
# Instantiating our network
network = create_network()

# Poutyne Model on GPU
model = Model(network, optimizer, loss_function, 
              batch_metrics=['accuracy'],
              device="all")

# Train
model.fit_generator(train_loader, valid_loader, epochs=num_epochs)

Epoch: 1/5 Train steps: 1500 Val steps: 375 17.00s loss: 0.364252 acc: 88.235417 val_loss: 0.085155 val_acc: 97.458333
Epoch: 2/5 Train steps: 1500 Val steps: 375 13.68s loss: 0.121587 acc: 96.331250 val_loss: 0.060322 val_acc: 98.183333
Epoch: 3/5 Train steps: 1500 Val steps: 375 13.66s loss: 0.094449 acc: 97.193750 val_loss: 0.053570 val_acc: 98.375000
Epoch: 4/5 Train steps: 1500 Val steps: 375 13.60s loss: 0.080947 acc: 97.633333 val_loss: 0.045832 val_acc: 98.700000
Epoch: 5/5 Train steps: 1500 Val steps: 375 13.82s loss: 0.072224 acc: 97.760417 val_loss: 0.042439 val_acc: 98.750000


[{'epoch': 1,
  'loss': 0.3642516027536864,
  'time': 17.00069876294583,
  'acc': 88.23541666666667,
  'val_loss': 0.08515502402248482,
  'val_acc': 97.45833333333333},
 {'epoch': 2,
  'loss': 0.12158718129402647,
  'time': 13.684815986081958,
  'acc': 96.33125,
  'val_loss': 0.06032236566596354,
  'val_acc': 98.18333333333334},
 {'epoch': 3,
  'loss': 0.09444938493948818,
  'time': 13.658660827204585,
  'acc': 97.19375,
  'val_loss': 0.05356967293145135,
  'val_acc': 98.375},
 {'epoch': 4,
  'loss': 0.08094733673124574,
  'time': 13.596006898209453,
  'acc': 97.63333333333334,
  'val_loss': 0.04583202180735922,
  'val_acc': 98.7},
 {'epoch': 5,
  'loss': 0.07222401596867713,
  'time': 13.818883433938026,
  'acc': 97.76041666666667,
  'val_loss': 0.04243871802262341,
  'val_acc': 98.75}]